<a href="https://colab.research.google.com/github/NeuroWinter/news-headline-ai/blob/master/Spacy_transfomers_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U spacy[cuda100]
!pip install spacy-transformers
!python3 -m spacy download en_trf_distilbertbaseuncased_lg
!pip install torch

     |████████████████████████████████| 10.4MB 1.2MB/s 
     |████████████████████████████████| 2.2MB 45.2MB/s 
     |████████████████████████████████| 3.7MB 46.0MB/s 
     |████████████████████████████████| 122kB 66.6MB/s 
     |████████████████████████████████| 382.9MB 43kB/s 
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 61kB 2.1MB/s 
     |████████████████████████████████| 296kB 8.1MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 870kB 65.6MB/s 
     |████████████████████████████

In [0]:
import pandas as pd
import csv

# REMEBER TO UPLOAD THIS!
input_file = 'news_headlines.csv'

TRAIN_DATA = []
df = pd.read_csv(input_file)

list_of_sites = list(df['news_site'].unique())

for row in df.iterrows():
    row_headline = row[1][2].strip()
    row_site     = row[1][1]
    if row_headline:
        # Add all the sites to the dict
        row_cat            = {site: 0.0 for site in list_of_sites}
        row_cat[row_site] += 1.0
        TRAIN_DATA.append((row_headline.strip(), {"cats": row_cat}))
print(TRAIN_DATA[0])
df = None

('Experts say adapting to climate change can pay off manifold', {'cats': {'ABC - Tech': 1.0, 'ABC - Top Stories': 0.0, 'ABC - US': 0.0, 'ABC - World': 0.0, 'Al Jazeera': 0.0, 'AllAfrica News - Mozambique': 0.0, 'ARRL': 0.0, 'Ars Technica - all features': 0.0, 'Ars Technica - all news': 0.0, 'Axios - Business': 0.0, 'Axios - Energy': 0.0, 'Axios - Future': 0.0, 'Axios - Health Care': 0.0, 'Axios - Politics': 0.0, 'Axios - Science': 0.0, 'Axios - Technology': 0.0, 'Axios - Top': 0.0, 'Axios - World': 0.0, 'Bandladesh - bdnews24.com': 0.0, 'Bangkok Post - Lifestyle': 0.0, 'Bangkok Post - Most Recent': 0.0, 'Bangkok Post - Top Stories': 0.0, 'Bangkok Post - Travel': 0.0, 'BBC': 0.0, 'BBC - Africa': 0.0, 'BBC - Asia': 0.0, 'BBC - England': 0.0, 'BBC - Europe': 0.0, 'BBC - Latin America': 0.0, 'BBC - Middle East': 0.0, 'BBC - Northern Ireland': 0.0, 'BBC - Scotland': 0.0, 'BBC - UK': 0.0, 'BBC - US & Canada': 0.0, 'BBC - Wales': 0.0, 'BBC - World': 0.0, 'Breitbart "News"': 0.0, 'Buzzfeed - U

In [0]:
# RUN THIS TO MAKE SURE THAT SPACY IS WORKING
import spacy

nlp = spacy.load('en_trf_distilbertbaseuncased_lg')
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)

In [0]:
TRAIN_DATA = [
    ("text1", {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0, "TESTING": 0.0}})
]

In [2]:
import spacy
import random
import torch

from spacy.util         import minibatch
from spacy_transformers import TransformersLanguage, TransformersWordPiecer, TransformersTok2Vec

BATCH_SIZE = 1000
ITERATIONS = 10
MODEL      = "distilbert-base-uncased"

is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("USING GPU")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

print(f"Number of lables: {len(list_of_sites)}")
print(f"Number of training data: {len(TRAIN_DATA)}")

nlp = spacy.load("en_trf_distilbertbaseuncased_lg")
print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
textnews = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True})
for label in list_of_sites:
    textnews.add_label(label)
nlp.add_pipe(textnews)
optimizer = nlp.resume_training()
for i in range(ITERATIONS):
    random.shuffle(TRAIN_DATA)
    losses = {}
    for x, batch in enumerate(minibatch(TRAIN_DATA, size=BATCH_SIZE)):
        texts, news = zip(*batch)
        nlp.update(texts, news, sgd=optimizer, losses=losses)
        print(f"Batch number: {x}, Losses: {losses}")
    print(f"Iteration: {i}, Losses: {losses}")

Batch number: 82, Losses: {'trf_textcat': 8.201778246075264e-05}
Batch number: 83, Losses: {'trf_textcat': 8.300654940285312e-05}
Batch number: 84, Losses: {'trf_textcat': 8.399612829634862e-05}
Batch number: 85, Losses: {'trf_textcat': 8.498405179580004e-05}
Batch number: 86, Losses: {'trf_textcat': 8.597229589213384e-05}
Batch number: 87, Losses: {'trf_textcat': 8.696170414168591e-05}
Batch number: 88, Losses: {'trf_textcat': 8.794966322511755e-05}
Batch number: 89, Losses: {'trf_textcat': 8.89373221752976e-05}
Batch number: 90, Losses: {'trf_textcat': 8.992387245143618e-05}
Batch number: 91, Losses: {'trf_textcat': 9.091244317005476e-05}
Batch number: 92, Losses: {'trf_textcat': 9.190130845126987e-05}
Batch number: 93, Losses: {'trf_textcat': 9.289042486670951e-05}
Batch number: 94, Losses: {'trf_textcat': 9.387882494138466e-05}
Batch number: 95, Losses: {'trf_textcat': 9.486634019140183e-05}
Batch number: 96, Losses: {'trf_textcat': 9.585418354163266e-05}
Batch number: 97, Losses: 

In [24]:
a = nlp("Experts say adapting to climate change can pay off manifold ".lower())
a.cats
for key, value in sorted(a.cats.items(), key=lambda item: item[1])[-3:]:
    print("%s: %s" % (key, value))
# Ok that doesnt work too well.....

Google News - Canada - Top Stories: 0.030641430988907814
Haaretz - WorldNews: 0.04528398439288139
Reddit - /r/politics: 0.04632045701146126
